In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)


In [5]:
df = pd.read_csv('processed_data.csv')
df.head()

,text,sentiment
0,come ask leave house sunday live eviction show move white room,2
1,gonna lie big brother mess isnt fair even fan,2
2,production team doesnt care bite people mental health sad cant begin imagine whats go head right whos military already battle anxiety,0
3,kayode one responsible show bore hms,2
4,video s fan protest bbnaijas eviction twist lagos,1


In [7]:
import nltk
import spacy
import pandas as pd
import seaborn as sns
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

ConnectionError: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

In [ ]:
nlp = spacy.load('en_core_web_md')


In [4]:
def get_noun(lines):
	tokenized = nltk.word_tokenize(lines)
	nouns = set([word for (word, pos) in nltk.pos_tag(tokenized) if(pos[:2] == 'NN')])
	return nouns

def get_similar_words(nouns, aspects):
	aspect_classes = {k: list() for k in aspects}
	#aspect_classes = {aspect: [aspect] for aspect in aspects}
	for noun in nouns:
		scores = list()
		for aspect in aspects:
			aspect_token = nlp(aspect)
			noun_token = nlp(noun)
			similarity_score = aspect_token.similarity(noun_token)
			scores.append(similarity_score)
		index = scores.index(max(scores))
		aspect_name = aspects[index]

		if max(scores)>0.50:
			value = aspect_classes[aspect_name]
			value.append(aspect_name)
			value.append(noun)
			aspect_classes[aspect_name] = list(set(value))

		else:
			pass
	return aspect_classes

def get_sentiment(aspect_classes, text):
    #sentiment_dict = {k:0 for k in aspect_classes}
    #sentiment_dict = {asp : {'positive': 0, 'negative': 0, 'neutral': 0} for asp in aspect_classes}
    sentiment_dict = {asp : {'positive': 0, 'negative': 0} for asp in aspect_classes}
    for aspect in aspect_classes:
        alt_names = aspect_classes[aspect]
        for name in alt_names:
            inputs = tokenizer(text, name, return_tensors="pt")

        with torch.inference_mode():
            outputs = model(**inputs)

        scores = F.softmax(outputs.logits[0], dim=-1)
        label_id = torch.argmax(scores).item()
        #sentiment_aspect[aspect] = (model.config.id2label[label_id], scores[label_id].item())
        sentiment = model.config.id2label[label_id].lower()

        # For positive and negative sentiments, update the sentiment_dict
        if sentiment in ['positive', 'negative'] :
            value = sentiment_dict[aspect][sentiment] + 1
            sentiment_dict[aspect][sentiment] = value

    return sentiment_dict

def compute(text, aspects):
	#preprocessing
	preprocess_text = text#preprocess(text)
	#get nouns
	noun_list = get_noun(preprocess_text)
	#get alternative names of aspects
	aspect_classes = get_similar_words(noun_list, aspects)
	#get sentiment
	sentiment_result = get_sentiment(aspect_classes, text)
	#sentiment_result = {'aspect 1': score, 'aspect 2': score, ...}
	return sentiment_result



def aspect_sentiment(aspects):
	reviews_content = df
	aspect_score = {asp : {'positive': 0, 'negative': 0} for asp in aspects}

	#compute scores for each tweet
	if reviews_content.to_dict():
		for text in reviews_content['text']:
			sentiment_result = compute(text, aspects)
			for result in sentiment_result:
				score = sentiment_result[result]
				if score>0:
					aspect_score[result]['positive'] = aspect_score[result]['positive'] + score
				elif score<0:
					aspect_score[result]['negative'] = aspect_score[result]['negative'] - score
				else:
					pass
	else:
		print('No data')


	result_list = [[k, 'positive', v['positive']] for k,v in aspect_score.items()]
	result_list.extend([[k, 'negative', v['negative']] for k,v in aspect_score.items()])

	#plot the bar plot across all aspects
	aspects_df = pd.DataFrame(result_list, columns= ['aspect', 'sentiment', 'score'])
	sns.barplot(x = 'aspect', y = 'score', hue='sentiment', data=aspects_df)


(32707, 6)